# # 웹스크랩

## - 소스 가져오기

In [1]:
import requests

In [2]:
res = requests.get('http://www.coupang.com/np/rocketdelivery?page=1')

## - 파싱하기

In [3]:
from bs4 import BeautifulSoup

In [4]:
# html.parser 및 lxml 2가지 사용 가능
soup = BeautifulSoup(res.text, 'html.parser')

In [5]:
# crom browser의 기능 활용
ul_tag = soup.select_one('#productList')

In [6]:
li_list = ul_tag.find_all('li')
len(li_list)

60

In [7]:
result_list = []

for li_tag in li_list:
    result_list.append([li_tag.select_one('a > dl > dd > div.name').text.strip(),
                        li_tag.select_one('a > dl > dd > div.price-area > div.price-wrap > div.price > em > strong').text.strip().replace(',', ''),
                        li_tag.select_one('a > dl > dd > div.other-info > div.rating-star > span.rating-total-count').text.strip()[1:-1] ])

result_list[:5]

[['오리온초코파이 정 딸기 앤 요거트, 444g', '4800', '117'],
 ['탐사수, 2L, 12개', '6300', '11431'],
 ['렌지셰프 한입치즈돈까스, 1kg', '12800', '21'],
 ['베베앙 아기물티슈 캡형, 100매, 10팩', '9900', '24950'],
 ['농심 봉지라면 4종 기획 세트, 20개입', '14900', '9088']]

In [8]:
import pandas as pd

In [9]:
df = pd.DataFrame(result_list, columns=['상품명', '가격', '추천도'])
df.head()

,상품명,가격,추천도
0,"오리온초코파이 정 딸기 앤 요거트, 444g",4800,117
1,"탐사수, 2L, 12개",6300,11431
2,"렌지셰프 한입치즈돈까스, 1kg",12800,21
3,"베베앙 아기물티슈 캡형, 100매, 10팩",9900,24950
4,"농심 봉지라면 4종 기획 세트, 20개입",14900,9088


In [10]:
df.to_csv('coupang_rocket.csv', index=False)

# # Open API 사용

## - 헤더정보 설정

In [2]:
header = {
    'X-Naver-Client-Id': 'Kt9JEg3vtZzC36UOTcyX',
    'X-Naver-Client-Secret': 'HrAlWytIVi'
}

## - 파라메터 : 검색어

In [4]:
data = {
    'query': '북한 여행',
    'display': 100
}

## - 호출

In [6]:
import requests

In [7]:
res = requests.get('https://openapi.naver.com/v1/search/blog.json',
                   headers=header, params=data)

## - 결과 데이터 파싱

In [13]:
import pandas as pd

In [15]:
result = res.json()

In [17]:
# 방법1. 리스트 만들어서 무식하게 처리
to_csv_list = []

for item in result['items']:
    to_csv_list.append([
        item['postdate'], item['title'],
        item['description'], item['bloggername']
    ])


df = pd.DataFrame(to_csv_list, columns=['날짜', '제목', '설명', '블로거'])
df.head()

,날짜,제목,설명,블로거
0,20171224,미국인의 <b>북한여행</b>기(희귀 사진),안녕하세요 오늘은 미국인의 <b>북한 여행</b>기라는 제목으로!북한의 모습을 한번...,김군의 여행스케치
1,20170810,미국 서점에서 본 <b>북한 여행</b>안내책,한국책들 사이에 있는 <b>북한 여행</b>책을 뽑아 들어봅니다.. 이 서점에는 딱...,예술미소
2,20170731,"미국, 8월 말부터 자국민 <b>북한여행</b> 전면금지 선언",미국 정부는 8월부터 <b>북한 여행</b> 전면 금지를 선언했는데요. 그 내막을 ...,"통일부 공식 블로그, '통일 미래의 꿈'"
3,20171205,&quot;<b>북한여행</b> 자유화로 평화통일 앞당기자!!&quot; : 435...,남한이 북한보다 우월한 체제라면 <b>북한여행</b> 자유화 당연하지 않은가?? 물...,sbrhob7979님의블로그
4,20171224,외국인과 신은미씨의 <b>북한 여행</b>기 비교,외국인과 신은미씨의 <b>북한 여행</b>기 비교 그녀만 좋았다는 <b>북한여행</...,Dean's Study


In [18]:
# JSON의 딕셔너리의 리스트는 바로 DataFrame으로 처리
df2 = pd.DataFrame(result['items'])
df2.head()

,bloggerlink,bloggername,description,link,postdate,title
0,http://blog.naver.com/8910a,김군의 여행스케치,안녕하세요 오늘은 미국인의 <b>북한 여행</b>기라는 제목으로!북한의 모습을 한번...,http://blog.naver.com/8910a?Redirect=Log&amp;l...,20171224,미국인의 <b>북한여행</b>기(희귀 사진)
1,http://blog.naver.com/ilchul2,예술미소,한국책들 사이에 있는 <b>북한 여행</b>책을 뽑아 들어봅니다.. 이 서점에는 딱...,http://blog.naver.com/ilchul2?Redirect=Log&amp...,20170810,미국 서점에서 본 <b>북한 여행</b>안내책
2,http://blog.naver.com/gounikorea,"통일부 공식 블로그, '통일 미래의 꿈'",미국 정부는 8월부터 <b>북한 여행</b> 전면 금지를 선언했는데요. 그 내막을 ...,http://blog.naver.com/gounikorea?Redirect=Log&...,20170731,"미국, 8월 말부터 자국민 <b>북한여행</b> 전면금지 선언"
3,http://blog.naver.com/sbrhob7979,sbrhob7979님의블로그,남한이 북한보다 우월한 체제라면 <b>북한여행</b> 자유화 당연하지 않은가?? 물...,http://blog.naver.com/sbrhob7979?Redirect=Log&...,20171205,&quot;<b>북한여행</b> 자유화로 평화통일 앞당기자!!&quot; : 435...
4,http://blog.naver.com/dean_cho,Dean's Study,외국인과 신은미씨의 <b>북한 여행</b>기 비교 그녀만 좋았다는 <b>북한여행</...,http://blog.naver.com/dean_cho?Redirect=Log&am...,20171224,외국인과 신은미씨의 <b>북한 여행</b>기 비교
